# ETF Screener and Watchlist Creator

This notebook implements an efficient, multi-step workflow to find ETFs based on specific criteria and automatically organize them into a new watchlist. It addresses the challenge of API rate limits by using bulk data fetching methods.

### Workflow:
1.  **Discover All ETF Epics:** We will navigate the market hierarchy to get a complete list of all available ETF epics.
2.  **Batch Fetch Details:** We will fetch the detailed data for all discovered ETFs in batches of 50 to minimize API calls.
3.  **Filter Locally:** We will load all the data into a pandas DataFrame and apply our desired filters locally (e.g., find ETFs that are `TRADEABLE` and have a `REGULAR` market mode).
4.  **Create Watchlist:** Finally, we will take the filtered list of epics and create a new, dedicated watchlist on Capital.com.

### 1. Setup and Imports

In [ ]:
import math
import os
import sys
from pathlib import Path

import pandas as pd
from loguru import logger

notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.capitalcom_bot.client_factory import get_client  # noqa: E402

logger.remove()
logger.add(sys.stderr, level="INFO")
pd.set_option("display.max_rows", 100)

### Step 1: Discover All ETF Epics

First, we need to get a list of all ETF epics. We assume ETFs are under a specific market navigation node. *Note: You may need to adjust the `etf_node_id` based on the output of `setup_api_workflow.ipynb`.*

In [ ]:
all_etf_epics = []
etf_node_id = "hierarchy_v1.etf_group.etf"  # This might need adjustment!

try:
    with get_client(demo_mode=True) as client:
        logger.info(f"Fetching all markets from node: {etf_node_id}")
        # The API might paginate, but for now we assume it returns all in one go.
        etf_group_content = client.get_markets_by_category(etf_node_id)
        if etf_group_content.get("markets"):
            all_etf_epics = [market["epic"] for market in etf_group_content["markets"]]
            logger.success(f"Discovered {len(all_etf_epics)} total ETF epics.")
        else:
            logger.warning("No markets found in the specified node.")
except Exception as e:
    logger.error(f"Failed to discover ETF epics: {e}", exc_info=True)

### Step 2: Batch Fetch Details for All ETFs

Now we use the powerful `epics` parameter of the `/markets` endpoint to fetch details in batches, avoiding thousands of individual API calls.

In [ ]:
all_etf_details = []
BATCH_SIZE = 50

if all_etf_epics:
    try:
        with get_client(demo_mode=True) as client:
            num_batches = math.ceil(len(all_etf_epics) / BATCH_SIZE)
            logger.info(f"Fetching details in {num_batches} batches of {BATCH_SIZE}...")

            for i in range(0, len(all_etf_epics), BATCH_SIZE):
                batch_epics = all_etf_epics[i : i + BATCH_SIZE]
                logger.debug(f"Fetching batch {i // BATCH_SIZE + 1}/{num_batches}...")

                response_model = client.get_markets_by_epics(batch_epics)

                # This ensures the dictionary keys match the original JSON field names (aliases).
                batch_details = [
                    detail.model_dump(by_alias=True)
                    for detail in response_model.market_details
                ]
                all_etf_details.extend(batch_details)

            logger.success(
                f"Successfully fetched details for {len(all_etf_details)} ETFs."
            )

    except Exception as e:
        logger.error(f"Failed during batch fetch: {e}", exc_info=True)

### Step 3: Load to DataFrame and Filter Locally

With all the data now in memory, we can use the power of pandas to perform complex filtering without any more API calls.

In [ ]:
filtered_epics = []

if all_etf_details:
    df_etfs = pd.json_normalize(all_etf_details)
    logger.info(f"Created DataFrame with {len(df_etfs)} ETFs.")

    tradeable_filter = df_etfs["snapshot.marketStatus"] == "TRADEABLE"

    def check_regular_mode(modes):
        if isinstance(modes, list):
            return "REGULAR" in modes
        return False

    regular_mode_filter = df_etfs["snapshot.marketModes"].apply(check_regular_mode)

    df_filtered = df_etfs[tradeable_filter & regular_mode_filter]

    logger.success(f"Found {len(df_filtered)} ETFs matching all criteria.")

    display_columns = [
        "instrument.epic",
        "instrument.name",
        "snapshot.marketStatus",
        "snapshot.marketModes",
    ]
    display(df_filtered[display_columns])

    filtered_epics = df_filtered["instrument.epic"].tolist()

### Step 4: Create a New Watchlist from the Filtered Results

Finally, we take our list of filtered epics and create a new watchlist on Capital.com for easy access.

In [ ]:
new_watchlist_name = "Tradeable ETFs"

if filtered_epics:
    try:
        with get_client(demo_mode=True) as client:
            # Clean up any old watchlist with the same name for a clean, repeatable run
            logger.info("Checking for existing watchlists to clean up...")
            existing_watchlists = client.get_watchlists()
            for wl in existing_watchlists.watchlists:
                if wl.name == new_watchlist_name:
                    logger.warning(
                        f"Deleting existing watchlist named '{new_watchlist_name}' (ID: {wl.id})."
                    )
                    client.delete_watchlist(wl.id)

            # Create the new watchlist with all our filtered epics in one go
            logger.info(
                f"Creating new watchlist '{new_watchlist_name}' with {len(filtered_epics)} ETFs..."
            )
            response = client.create_watchlist(
                name=new_watchlist_name, epics=filtered_epics
            )

            logger.success(
                f"Successfully created watchlist '{new_watchlist_name}' with ID {response.watchlist_id}."
            )
            print(
                "\nYou can now check your Capital.com account to see the new watchlist!"
            )

    except Exception as e:
        logger.error(f"Failed to create watchlist: {e}", exc_info=True)
else:
    logger.info("No ETFs to add to a watchlist based on the filters.")